In [ ]:
## 1
## d'abord on raboute les fichiers fasta références NCBI et ENA
cd /home/moustache/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S
gzip -d ./ncbi/ncbi_mitochondrion_AND_16S.fasta.gz
cat ./ncbi/ncbi_mitochondrion_AND_16S.fasta ./ena/ENA_Amphibiens_BZH_16Smito.fasta ./ena/ENA_Amphibiens_invasifs_CROAA_16S.fasta | gzip > ./ena_ncbi/NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA_16S.fasta.gz
gzip ./ncbi/ncbi_mitochondrion_AND_16S.fasta

In [2]:
## 2
## on élimine les séquences dupliquées
cd /home/moustache/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/ena_ncbi

gzip NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA_16S.fasta
seqkit rmdup -s NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA_16S.fasta.gz > NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA_16S_nodup.fasta.gz

[INFO] 1842392 duplicated records removed


In [5]:
## 3
## on extrait les amplicons des différentes bases fasta (NCBI et ENA) à partir des amorces Amph16S
cd /home/moustache/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/ena_ncbi

seqkit amplicon NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA_16S_nodup.fasta.gz \
    --threads 8 -F ACGAGAAGACCC..TGGA.CTT -R ATCCAACATCGAGGTCGTAA -r 1:-1 --max-mismatch 4 > NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.fasta
grep ">" NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.fasta | wc -l # 10642 sequences references
rm NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA_16S_nodup.fasta.gz

[INFO] 1 primer pair loaded
10642


In [1]:
## 4
## ensuite on extrait les identifiants (Accession) NCBI et ENA à partir du fasta des séquences références
cd /home/moustache/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/ena_ncbi

grep "^>" NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.fasta  | awk -F"_gene" '/\|/ {print $1}' | sed -e "s/>//" | awk '{print substr($0,index($0," | ")+5)}' | awk -F'|' '{print $1}' \
    > IDs_NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.txt


In [15]:
## 5
## installer taxonkit à partir de conda
# https://bioinf.shenwei.me/taxonkit/tutorial
## mise en place de la base de concordance entre NCBI Accessions et Limeages
mkdir ~/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/taxdump/
cd ~/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/taxdump/
wget ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz
tar -xvzf taxdump.tar.gz
cd ./taxdump
cp names.dmp nodes.dmp delnodes.dmp merged.dmp ~/.taxonkit

>=@
>=@
>=@
>=@
>mkdir: cannot create directory ‘/Users/tonyrobinet/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/taxdump/’: File exists
=@
>=@
>dyld[24525]: Library not loaded: @rpath/libintl.8.dylib
  Referenced from: <0714F760-4165-3CE5-82EB-B7E0AC670F9D> /opt/miniconda3/bin/wget
  Reason: tried: '/opt/miniconda3/bin/../lib/libintl.8.dylib' (no such file), '/opt/miniconda3/bin/../lib/libintl.8.dylib' (no such file), '/usr/local/lib/libintl.8.dylib' (no such file), '/usr/lib/libintl.8.dylib' (no such file, not in dyld cache)
Abort trap: 6
=@
>tar: Error opening archive: Failed to open 'taxdump.tar.gz'
=@
>bash: cd: ./taxdump: No such file or directory
=@
>cp: cannot stat 'names.dmp': No such file or directory
cp: cannot stat 'nodes.dmp': No such file or directory
cp: cannot stat 'delnodes.dmp': No such file or directory
cp: cannot stat 'merged.dmp': No such file or directory
=@


: 1

In [24]:
# autre méthode : https://www.biostars.org/p/330313/

bash: /home/moustache/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/ncbi: est un dossier
--2025-05-07 11:58:24--  ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz
           => ‘taxdump.tar.gz.1’
Résolution de ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)… 130.14.250.7, 130.14.250.12, 130.14.250.11, ...
Connexion à ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)|130.14.250.7|:21… connecté.
Ouverture de session en tant que anonymous… Session établie.
==> SYST ... terminé.    ==> PWD ... terminé.
==> TYPE I ... terminé.  ==> CWD (1) /pub/taxonomy ... terminé.
==> SIZE taxdump.tar.gz ... 69612552
==> PASV ... terminé.    ==> RETR taxdump.tar.gz ... terminé.
Taille : 69612552 (66M) (non certifiée)

taxdump.tar.gz.1    100%[===================>]  66,39M  11,8MB/s    ds 6,5s    

2025-05-07 11:58:35 (10,2 MB/s) - ‘taxdump.tar.gz.1’ enregistré [69612552]



: 1

In [ ]:
## 6
## On établit la correspondance entre l'accession NCBI et TaxId
## en générant un 1er fichier de correspondances [NCBI_Accession TaxId]
#### cette étape peut durer plusieurs heures (2 000 TaxId par heure)

cd /home/moustache/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/ena_ncbi

#file="IDs_ncbi_mitochondrion_AND_1070F_1340R.txt"
file="IDs_NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.txt"

while IFS= read -r line; do
  efetch -db nuccore -id "$line" -format docsum | xtract -pattern DocumentSummary -element Caption,TaxId >> NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_avec_extension.txt
done < "$file"


In [1]:
cd ~/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/ena_ncbi
wc -l NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.txt # 10642

>=@
>10642 NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.txt
=@


: 1

In [5]:
## 7
## On crée ensuite un fichier [NCBI_Accession Lineage_7ranks] à partir d'un fichier de correspondances [NCBI_Accession TaxId] par l'outil taxonkit
## ce fichier servira pour éablir le fasta final, avec uniquement [Lineage_7ranks] en header.

cd ~/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/ena_ncbi
# -element Caption,TaxId # pour avoir la correspondance avec les accessions NCBI

## cette commande fonctionne et reformatte correctement en 7 rangs taxo
#cat Taxonkit_test.txt | taxonkit lineage -i 2 | taxonkit reformat2 -I 2 -f "{domain};{phylum};{class};{order};{family};{genus};{species}" | cut -f 1,4 > Taxonkit_test_reformatted.txt

cat NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.txt | taxonkit lineage -i 2 | taxonkit reformat2 -I 2 -f "{domain};{phylum};{class};{order};{family};{genus};{species}" \
    | cut -f 1,4 > NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA_1070F_1340R_reformatted.txt


>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>10:23:37.520 [ERRO] taxonomy data not found, please download and uncompress ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz, and copy "names.dmp", "nodes.dmp", "delnodes.dmp", and "merged.dmp" to /Users/tonyrobinet/.taxonkit
10:23:37.520 [ERRO] taxonomy data not found, please download and uncompress ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz, and copy "names.dmp", "nodes.dmp", "delnodes.dmp", and "merged.dmp" to /Users/tonyrobinet/.taxonkit
cat: write error: Broken pipe
=@


: 1

In [ ]:
## 8
## ENFIN on reformate le fasta de références avec en entête uniquement les 7 rangs de classification taxonomique
## "{domain};{phylum};{class};{order};{family};{genus};{species}"
# c.a.d. que dans ce nouveau fasta de références, on remplace l'entête par le lineage, i.e. >Bacteria;Pseudomonadota;Gammaproteobacteria;Enterobacterales;Vibrionaceae;Vibrio;vibrio
# utilisable par dada2 pour l'assignation taxo


#sed -i -e 's/lcl|//g' ncbi_mitochondrion_AND_1070F_1340R.fasta
#cat Taxonkit_test_sequences.fa | awk -F"." '/\>/ {print $1}' > Taxonkit_test_sequences_reform.fa
#seqkit replace -k Taxonkit_test_reformatted.txt -p '(\S+)' -r ' {kv}' Taxonkit_test_sequences_reform.fa | sed -r 's/> />/g'

# on ne garde que NCBI_Accession en header du fasta
#cat ncbi_mitochondrion_AND_1070F_1340R.fasta | awk -F"." '/\>/ {print $1}' > ncbi_mitochondrion_AND_1070F_1340R_reform.fasta

# on remplace NCBI_Accession par Lineage_7ranks en header du fasta
#seqkit replace -k TaxId_ncbi_mitochondrion_AND_1070F_1340R_reformatted.txt -p '(\S+)' -r ' {kv}' ncbi_mitochondrion_AND_1070F_1340R_reform.fasta \
#    | sed -r 's/> />/g' > ncbi_mitochondrion_AND_1070F_1340R_ready.fasta


cd ~/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/ena_ncbi

cat NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.fasta | awk 'BEGIN { FS=OFS="_gene" } /^>/ { print $1; next }1' \
    | awk 'BEGIN { FS=OFS="|" } /^>/ { print ">" $2; next }1' > NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_reform.fasta

seqkit replace -k NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA_1070F_1340R_reformatted.txt -p '(\S+)' -r ' {kv}' NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_reform.fasta \
    | sed -r 's/> />/g' | gzip > NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignTaxonomy.fasta.gz

# [INFO] read key-value file: NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA_1070F_1340R_reformatted.txt
# [INFO] 10638 pairs of key-value loaded
# le fichier xxx_ready.fasta est prêt à l'assignation taxo par dada2

>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>=@
>>NC_082172.1
ACGAGAAGACCCTGTGGAACTTCAAAATCAATAGCCACCACACACCCAACCACAAACCTA
CTGGCCCACCACCAAGCAACGCTGGCTAATATTTTTAGGTTGGGGCGACCTTGGAGAAAA
ACAGACCCTCCAAAATCAAGGCCACATCCCTTAACCAAGAGCCACCCCTCAACGTACCAA
CAGTGTTCAGACCCAATAAAATTGACCAATGGACCAAGCTACCCCAGGGATAACAGCGCA
ATCTCCCCCAAGAGCCCCTATCGACGGGGAGGTTTACGACCTCGATGTTGGAT
>NC_082171.1
ACGAGAAGACCCTGTGGAACTTCAAAATTAATAGCCACCCCACCCAACTCACAAACCCAC
TAACGGGCCCATCACTCAAGACACTGGCTAACATTTTTAGGTTGGGGCGACCTTGGAGAA
AAACAGATCCTCCAAAAATAAGGCCAAATCCCTTAACCAAGAGCAACCCCTCAACGTACT
AACAGTACCCAGACCCAGTATATCTGACCAATGAACCAAGCTACCCCAGGGATAACAGCG
CAATCTCCCCCAAGAGCCCCTATCGACGAGGAGGTTTACGACCTCGATGTTGGAT
>NC_082170.1
ACGAGAAGACCCTGTGGAACTTCAAAATCAATAGCCACCCCACCCAACCCACAAACCCAC
CCAGGCCCACCACCCAAAACACTGGCTAACATTTTTAGGTTGGGGCGACCTTGGAGAAAA
ACAGATCCTCCAAAAATAGGGCTAAATCCCCTAACCGAGAGCAACCTCTCAACGTGCTAA
CAGCACCCAGACCCAGTAAATCTGACCAATGAACCAAGCTACCCCA

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



ATGAATATTTTAATTTTTTTGTTGGGGTGACATGGAGATATATTTAACTCTTTATTTTAT
TTTTCAATTATTATTGTTTATTTGACCTTTTATTATTGTTATAAGATATAGATACCTTAG
GGATAACAGCGTAATTATTTTTGAGAGTTCATATTGACGAAATAGATTGCGACCTCGATG
TTGGAT
>NC_080489.1
ACGAGAAGACCCTATAGATCTTTATTTTTTATTTGTCAATTATATTTTTGGTTTAATGTC
TATTTTGATTTATTTGATTGTTTTGTTGGGGTGACATGGAGATATAATTTAAACTCTTCA
TTTTTGTTTTCATTGATTTATGTTTATTTGACCTTTTATTATTGTTTTAAGATTTAGATA
CCTTAGGGATAACAGCGTTATTTTTCTTGAGAGTTCGTATCGACAGATAAGATTGCGACC
TCGATGTTGGAT
>ON920960.1
ACGAGAAGACCCTATAGATCTTTATAAATAATTGTGTAATTATTTTTATATTAACACTTT
TATTCAATTATTTATTTTGTTGGGGTGACAAAAACATATAAATAACTTTTTTATTTACAA
AACATTGATTTATGAATATTAGATCCATTATTATTGATTATTAGAATAAGTTACCTTAGG
GATAACAGCGTAATCCTCTTCCAGAGTTCATATTTACGAGGGGGTTTGCGACCTCGATGT
TGGAT
>ON380414.1
ACGAGAAGACCCTATAGATCTTCACTCTGAGAGAGTTTAGTTGGGGCAACTGAAAGATTA
AATTCTTTCTTAATTAAACTTTTCAAGAGAAGAGCTGACCCCCCAGGATCAAAGGAAAAG
ATACCTTAGGGTTAACAGCACAATAAAGGTAATGGAGATCTAATCTGTTTCCTAGCTTGT
GACCTCGATGTTGAAT
>ON204017.1
ACGAGAAGACCCTATAGATCTTTACTTATAAATATTTCTGTTAGCTCTATTTAA

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



AAAAGTAACGAAGGCGTACACAGGGTAAAGATATAATAGTATAAAGCCTGACTGTAAAGA
TGACAATCTGAGCAGGGCCTATATGGGCGGTTATAATGACCCGTTAATATAGAGTGAAAT
AGTTAACGATCAACGGATAAAAGTTACCATGGGGATAACAGCGTAATATCGTTCAAGAGT
TCATATCGACGACGATGTTTGCGACCTCGATGTTGAAT
>NC_014886.1
ACGAGAAGACCCCATTGAGCTTTACTGGAAACTTATACTGTTCACATCAAATTAATTGTA
TTACTGGTGGGAAAGATGCATATAGCACTCTAAAACCACTCTTTAATTTTGAGGGGAACT
TAGACGGTAAGACCGACGAAACGTGTAAGTTGGACAGTTTGGTTGGGGCGACCGCCTTCT
AAAAAGTAACGAAGGCGTACATAAGGTAAAGATATAATAGTATAAAGCCTGACTGTAAAG
ATGACAATCTGAGCAGAGCCTATATGGGCGGTTATAATGATCCGTTAATATAGAGTGAAA
TAATTAACGATCAACGGATAAAAGTTACCATGGGGATAACAGCGTAATATCGTTCAAGAG
TTCATATCGACGACGATGTTTGCGACCTCGATGTTGAAT
>NC_014885.1
ACGAGAAGACCCTATTGAGCTTTACTGGAAACTTATATAGTTAGAAAAAAAAANTTGTGT
GTGGGAGTGTAATGCTCTTAAAGTAATCACNTTTTTTTATTTATAATAACTTATGGGCAA
AGCCTAAAAAGGTGTAAGTAGGACAGTTTGGTTGGGGCAACCGCCTTCTAAAAAGTAACG
AAGGTGAACATAAGGTAAAGACATAATGATGTAATAAGCCTGACTGTAAAGAAAACAATC
TGAGCAGGGCCCATTAGGGCGGTTAAAATGACCCGTTAATATAAAGTGTAATTGTTAACG
ATCAACGGATAAAAGTTACCATAGGGATAACAGCGTAATAT

In [1]:
## 9
## Assignation à l'espèce par dada2::assignSpecies()
## il faut un fasta référence de format >SeqID genus species ACGAATGTGAAGTAA......

cd /home/moustache/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/ena_ncbi
ls -la

# Définir les chemins des fichiers
metadata_file="NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA_1070F_1340R_reformatted.txt"
echo $"Décompression du fasta assignTaxonomy"
cat NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignTaxonomy.fasta.gz | gzip -d > NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignTaxonomy.fasta
fasta_file="NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignTaxonomy.fasta"
rm NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignSpecies.fasta # élimine toute éventuelle sortie précédente
touch NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignSpecies.fasta # crée une nouvelle sortie vide

# Extraire les SeqID et les Genus Species
grep "^>" "$fasta_file" | awk -F" " '{print $1}' | tr -d '>' | cut -d ';' -f 7 > gen_list.txt
grep "^>" "$fasta_file" | awk -F" " '{print $2}' > sp_list.txt
awk -F"\t" '{print $1}' "$metadata_file" > seqid.txt

# Extraire les séquences
awk '/^>/ {if (seq) print seq; seq=""; next} {seq=seq $0} END {if (seq) print seq}' "$fasta_file" > sequences.txt

#Concaténer les SeqID, Genus Species et les séquences
paste seqid.txt gen_list.txt sp_list.txt sequences.txt | while IFS=$'\t' read -r id gen sp sequence; do
  echo ">$id $gen $sp" >> "NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignSpecies.fasta"
  echo "$sequence" >> "NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignSpecies.fasta"
done

echo $"assignTaxonomy: nb of sequences" ; grep ">" "NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignTaxonomy.fasta" | wc -l
echo $"assignSpecies: nb of sequences" ; grep ">" "NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignSpecies.fasta" | wc -l

echo $"Compression des fasta"
cat "NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignSpecies.fasta" | gzip >  "NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignSpecies.fasta.gz"
cat "NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignTaxonomy.fasta" | gzip >  "NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignTaxonomy.fasta.gz"

echo $"Supression des fichiers intermédiaires"
rm seqid.txt gen_list.txt sp_list.txt sequences.txt
rm NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignSpecies.fasta NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignTaxonomy.fasta

echo $"Terminé"

total 433508
drwxrwxr-x 2 moustache moustache      4096 mai   14 11:02 .
drwxr-xr-x 6 moustache moustache      4096 mai   10 10:25 ..
-rw-r--r-- 1 moustache moustache      6148 mai   10 14:56 .DS_Store
-rw-rw-r-- 1 moustache moustache   3998026 mai   13 16:26 formatted_fasta.fasta
-rw-rw-r-- 1 moustache moustache    121466 mai   10 14:34 IDs_NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.txt
-rw-rw-r-- 1 moustache moustache     13576 mai   10 14:48 NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_avec_extension.txt
-r--r--r-- 1 moustache moustache    981288 mai   10 10:37 NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA_1070F_1340R_reformatted.txt
-r--r--r-- 1 moustache moustache    175300 mai    9 19:22 NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.txt
-rw-rw-r-- 1 moustache moustache   3384865 mai   14 11:02 NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignSpecies.fasta
-rw-rw-r-- 1 moustache moustache    573907 mai   14 11:02 NCBI_ENA_Amphi

In [1]:
cd /home/moustache/sync/BioLav/M2_Ivan/Stage_M2/bioinfo/RefBase_Amph16S/ena_ncbi
gzip "NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignTaxonomy.fasta"
ls -la

total 432904
drwxrwxr-x 2 moustache moustache      4096 mai   14 10:58 .
drwxr-xr-x 6 moustache moustache      4096 mai   10 10:25 ..
-rw-r--r-- 1 moustache moustache      6148 mai   10 14:56 .DS_Store
-rw-rw-r-- 1 moustache moustache   3998026 mai   13 16:26 formatted_fasta.fasta
-rw-rw-r-- 1 moustache moustache    108758 mai   14 10:56 gen_list.txt
-rw-rw-r-- 1 moustache moustache    121466 mai   10 14:34 IDs_NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.txt
-rw-rw-r-- 1 moustache moustache     13576 mai   10 14:48 NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_avec_extension.txt
-r--r--r-- 1 moustache moustache    981288 mai   10 10:37 NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA_1070F_1340R_reformatted.txt
-r--r--r-- 1 moustache moustache    175300 mai    9 19:22 NCBIACC_TaxId_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R.txt
-rw-rw-r-- 1 moustache moustache   3384865 mai   14 10:56 NCBI_ENA_Amphibiens_BZH_ET_invasifs_CROAA__1070F_1340R_assignSpecies.fasta
-